In [1]:
from pwn import * 
from json import * 
from ast import literal_eval
from Crypto.Util.number import bytes_to_long
from sage.all import matrix
from typing import List, Tuple
import math, operator
from collections.abc import Sequence

# https://github.com/nneonneo/pwn-stuff/blob/main/math/solvelinmod.py

def _process_linear_equations(equations, vars, guesses) -> List[Tuple[List[int], int, int]]:
    result = []
    for rel, m in equations:
        op = rel.operator()
        if op is not operator.eq:
            raise TypeError(f"relation {rel}: not an equality relation")
        expr = (rel - rel.rhs()).lhs().expand()
        coeffs = [int(expr.coefficient(var)) % m for var in vars]
        const = int(expr.subs({var: guesses[var] for var in vars})) % m
        result.append((coeffs, const, m))
    return result

def solve_linear_mod(equations, bounds, verbose=False, **lll_args):
    vars = list(bounds)
    guesses = {}
    var_scale = {var: max(xmax - xmin, xmin - xmax, 1) for var, (xmin, xmax) in bounds.items()}
    for var in vars:
        xmin, xmax = bounds[var]
        guesses[var] = (xmax - xmin) // 2 + xmin

    var_bits = math.log2(int(prod(var_scale.values()))) + len(vars)
    mod_bits = math.log2(int(prod(m for _, m in equations)))
    if verbose:
        print(f"verbose: variable entropy: {var_bits:.2f} bits")
        print(f"verbose: modulus entropy: {mod_bits:.2f} bits")

    equation_coeffs = _process_linear_equations(equations, vars, guesses)
    is_inhom = any(const != 0 for _, const, _ in equation_coeffs)
    NR = len(equation_coeffs)
    NV = len(vars) + (1 if is_inhom else 0)

    B = matrix(ZZ, NR + NV, NR + NV)
    S = max(var_scale.values())
    eqS = S << (NR + NV + 1)
    if var_bits > mod_bits:
        eqS <<= int((var_bits - mod_bits) / NR) + 1

    col_scales = []
    for ri, (coeffs, const, m) in enumerate(equation_coeffs):
        for vi, c in enumerate(coeffs):
            B[NR + vi, ri] = c
        if is_inhom:
            B[NR + NV - 1, ri] = const
        col_scales.append(eqS)
        B[ri, ri] = m

    for vi, var in enumerate(vars):
        col_scales.append(S // var_scale[var])
        B[NR + vi, NR + vi] = 1

    if is_inhom:
        col_scales.append(S)
        B[NR + NV - 1, -1] = 1

    if verbose:
        print("verbose: scaling shifts:", [math.log2(int(s)) for s in col_scales])
        print("verbose: unscaled matrix before:")
        print(B.n())

    for i, s in enumerate(col_scales):
        B[:, i] *= s
    B = B.LLL(**lll_args)
    for i, s in enumerate(col_scales):
        B[:, i] /= s

    for i in range(B.nrows()):
        if sum(x < 0 for x in B[i, :]) > sum(x > 0 for x in B[i, :]):
            B[i, :] *= -1
        if is_inhom and B[i, -1] < 0:
            B[i, :] *= -1

    if verbose:
        print("verbose: unscaled matrix after:")
        print(B.n())

    for row in B:
        if any(x != 0 for x in row[:NR]):
            continue
        if is_inhom and row[-1] != 1:
            continue

        res = {var: row[NR + vi] + guesses[var] for vi, var in enumerate(vars)}
        return res

io = remote('157.15.86.73', 2004)
io.recvuntil(b'p = ')
p = int(io.recvline()[:-1].decode())

io.recvuntil(b'points = ')
points = literal_eval(io.recvline()[:-1].decode())

scale = [2**i for i in range(56, -1, -8)]

x = [i[0] for i in points]
b_values = [i[1] for i in points]

A_values = []
for i in x:
    multi_tmp = [(i ^ z) % p for z in range(10, -1, -1)]
    multi = [(o * f) % p for o in multi_tmp for f in scale]
    A_values.append(multi)

a = [var(f"a_{i}") for i in range(len(A_values[0]))]

eqs = []
for x, b in zip(A_values, b_values):
    eq = sum(c * d for c, d in zip(x, a))
    eqs.append((b == eq, p))

bounds = {x: (48, 70) for x in a}
print(bounds)

sol = solve_linear_mod(eqs, bounds)
ks = list(sol.values())[-8:]
print(ks)
print(sol.values())

y = bytes_to_long(bytes(ks))
io.sendline(str(y).encode())
io.recvline()

[x] Opening connection to 157.15.86.73 on port 2004
[x] Opening connection to 157.15.86.73 on port 2004: Trying 157.15.86.73
[+] Opening connection to 157.15.86.73 on port 2004: Done
{a_0: (48, 70), a_1: (48, 70), a_2: (48, 70), a_3: (48, 70), a_4: (48, 70), a_5: (48, 70), a_6: (48, 70), a_7: (48, 70), a_8: (48, 70), a_9: (48, 70), a_10: (48, 70), a_11: (48, 70), a_12: (48, 70), a_13: (48, 70), a_14: (48, 70), a_15: (48, 70), a_16: (48, 70), a_17: (48, 70), a_18: (48, 70), a_19: (48, 70), a_20: (48, 70), a_21: (48, 70), a_22: (48, 70), a_23: (48, 70), a_24: (48, 70), a_25: (48, 70), a_26: (48, 70), a_27: (48, 70), a_28: (48, 70), a_29: (48, 70), a_30: (48, 70), a_31: (48, 70), a_32: (48, 70), a_33: (48, 70), a_34: (48, 70), a_35: (48, 70), a_36: (48, 70), a_37: (48, 70), a_38: (48, 70), a_39: (48, 70), a_40: (48, 70), a_41: (48, 70), a_42: (48, 70), a_43: (48, 70), a_44: (48, 70), a_45: (48, 70), a_46: (48, 70), a_47: (48, 70), a_48: (48, 70), a_49: (48, 70), a_50: (48, 70), a_51: (48,

b"What's our secret ? Cool!! You can deal with hard challenge, get your treasure here: KMACTF{Us1nG_LLL_1s_4n_4rt_f0rm__:)))}\n"